<a href="https://colab.research.google.com/github/cacheop/facemask-detection/blob/master/Mask_prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

### Mount Google Drive

In [54]:
from datetime import datetime

now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y--%H-%M-%S")
print("date and time:",date_time)	

date and time: 06-15-2020--21-13-53


In [55]:
import tensorflow as tf
import os
from psutil import virtual_memory
import os
import cv2
import matplotlib.pyplot as plt
import xmltodict
import random
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm, trange

# Split data folder into Test and Train


In [56]:
from random import choice
import os
import shutil
from glob import glob                                                           
import cv2 

#setup dir names
HOME_DIR = "data/"
img_path = HOME_DIR + 'images/' #dir with images 
xml_path = HOME_DIR + 'labels/' #dir with annotations 

train_path = HOME_DIR + 'train/'
train_path_images = train_path + 'images/'
train_path_xml = train_path + 'labels/'
train_path_csv = train_path + 'csv/'

test_path = HOME_DIR + 'test/'
test_path_images = test_path + 'images/'
test_path_xml = test_path + 'labels/'
test_path_csv = test_path + 'csv/'

In [57]:
pngs = glob(img_path +'*.png')
jpegs = glob(img_path +'*.jpeg')
JPGs = glob(img_path +'*.JPG')

print("Converting pngs...")
for j in pngs:
    img = cv2.imread(j)
    cv2.imwrite(j[:-3] + 'jpg', img)
    print(j)
    os.remove(j)

print("Renaming jpeg to jpg...")
for name in jpegs:
    new_name = name[:-5] +'.jpg'
    print(name, new_name)
    os.rename(name, new_name)


print("Renaming JPG to jpg...")
for name in JPGs:
    new_name = name[:-4] +'.jpg'
    print(name, new_name)
    os.rename(name, new_name)

# TO-DO: fix XML reference from PNG and JPEG to JPG

Converting pngs...
Renaming jpeg to jpg...
Renaming JPG to jpg...
data/images\0450908675_50159485_mutation-virus-chine-inquietude.jpg data/images\0450908675_50159485_mutation-virus-chine-inquietude.jpg
data/images\190327-05_1024.jpg data/images\190327-05_1024.jpg
data/images\20200129001040.jpg data/images\20200129001040.jpg
data/images\20200206132618-c9fcbba8.jpg data/images\20200206132618-c9fcbba8.jpg
data/images\2bd1431c-e585-4faf-b54c-0c25ae00d725.jpg data/images\2bd1431c-e585-4faf-b54c-0c25ae00d725.jpg
data/images\89c93a24-3f73-11ea-a16e-39b824591591_image_hires_211659.jpg data/images\89c93a24-3f73-11ea-a16e-39b824591591_image_hires_211659.jpg
data/images\b3907fd44a737877c847748438b72de9size_1000x561_size.jpg data/images\b3907fd44a737877c847748438b72de9size_1000x561_size.jpg
data/images\organizacion-preocupacion-potencial-virus-propague.jpg data/images\organizacion-preocupacion-potencial-virus-propague.jpg


In [58]:
#arrays to store file names
imgs = []
xmls = []

#split ratio 
train_ratio = 0.8
test_ratio = 0.1

#total count of imgs + annotations
total_img_count = len(os.listdir(img_path))/2
total_xml_count = len(os.listdir(xml_path))/2
print ('There are', total_img_count, 'images and ', total_xml_count, 'annotations.')

#they gotta be the same count
if (total_img_count != total_img_count):
    print ("WARNING: mismatch!")

There are 4.0 images and  4.0 annotations.


In [59]:
import os.path

not_paired = 0

#sorting files to corresponding arrays
for (dirname, dirs, files) in os.walk(xml_path):
    for filename in files:
        img_file_pair = img_path + filename[:-4] + '.jpg'
        if os.path.isfile(img_file_pair):
            xmls.append(filename)
        else:
            print("will remove", filename)
            not_paired += 1
            os.remove(dirname+filename)
print('Removed ', not_paired, 'not paired XML files for good')

for (dirname, dirs, files) in os.walk(img_path):
    for filename in files:
        imgs.append(filename)

total_imgs = len(imgs)
total_xmls = len(xmls)
print('Images', total_imgs)
print('XML files', total_xmls)

#counting range for cycles

count_for_train = int(len(imgs)*train_ratio)
count_for_test = int(len(imgs)*test_ratio)

print("Split files: ", count_for_train, count_for_test)

#cycle for train dir images
for x in range(count_for_train):
    file_jpg = choice(imgs) # get name of random image from origin dir
    file_xml = file_jpg[:-4] +'.xml' # get name of corresponding annotation file
    print("Train:    [", file_jpg,"]", "\n[", file_xml,"]\n\n")

    #move both files into train dir
    shutil.move(os.path.join(img_path, file_jpg), os.path.join(train_path_images, file_jpg))
    imgs.remove(file_jpg)

    shutil.move(os.path.join(xml_path, file_xml), os.path.join(train_path_xml, file_xml))
    xmls.remove(file_xml)

#cycle for test dir   
for x in range(count_for_test):

    file_jpg = choice(imgs) # get name of random image from origin dir
    file_xml = file_jpg[:-4] +'.xml' # get name of corresponding annotation file
    print("Test:  [", file_jpg,"]", "\n[", file_xml,"]\n\n")


    #move both files into test dir
    shutil.move(os.path.join(img_path, file_jpg), os.path.join(test_path_images, file_jpg))
    imgs.remove(file_jpg)

    shutil.move(os.path.join(xml_path, file_xml), os.path.join(test_path_xml, file_xml))
    xmls.remove(file_xml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)

#summary information after splitting
print('Total images: ', total_img_count)

Removed  0 not paired XML files for good
Images 8
XML files 8
Split files:  6 0
Train:    [ 0450908675_50159485_mutation-virus-chine-inquietude.jpg ] 
[ 0450908675_50159485_mutation-virus-chine-inquietude.xml ]


Train:    [ 2bd1431c-e585-4faf-b54c-0c25ae00d725.jpg ] 
[ 2bd1431c-e585-4faf-b54c-0c25ae00d725.xml ]


Train:    [ 89c93a24-3f73-11ea-a16e-39b824591591_image_hires_211659.jpg ] 
[ 89c93a24-3f73-11ea-a16e-39b824591591_image_hires_211659.xml ]


Train:    [ organizacion-preocupacion-potencial-virus-propague.jpg ] 
[ organizacion-preocupacion-potencial-virus-propague.xml ]


Train:    [ 20200129001040.jpg ] 
[ 20200129001040.xml ]


Train:    [ 190327-05_1024.jpg ] 
[ 190327-05_1024.xml ]


Total images:  4.0


In [60]:
print('Images in train dir:', len(os.listdir(train_path_images)))
print('Images in test dir:', len(os.listdir(test_path_images)))
print('XMLs in train dir:', len(os.listdir(train_path_xml)))
print('XMLs in test dir:', len(os.listdir(test_path_xml)))

Images in train dir: 603
Images in test dir: 73
XMLs in train dir: 603
XMLs in test dir: 73


# XML to CSV

In [61]:
from glob import glob                                                           
from tqdm import tqdm, trange
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

def xml_to_csv(path):
    print("------------->", path)
    xml_list = []
    for xml_file in tqdm(glob(path + '/*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [62]:
print('Converting XMLs in folder: ', train_path_xml, '...\n')
train_xml_df = xml_to_csv(train_path_xml)
train_xml_df.head()


 44%|██████████████████████████████████                                            | 263/603 [00:00<00:00, 2239.33it/s]

Converting XMLs in folder:  data/train/labels/ ...

-------------> data/train/labels/


100%|██████████████████████████████████████████████████████████████████████████████| 603/603 [00:00<00:00, 3131.45it/s]


,filename,width,height,class,xmin,ymin,xmax,ymax
0,-1x-1.jpg,2000,1363,good,10,750,252,1164
1,-1x-1.jpg,2000,1363,bad,805,43,886,133
2,-1x-1.jpg,2000,1363,good,584,34,700,129
3,-1x-1.jpg,2000,1363,good,52,1,160,56
4,-1x-1.jpg,2000,1363,good,461,181,569,292


In [63]:
print('Converting XMLs in folder: ', test_path_xml, '...\n')
test_xml_df = xml_to_csv(test_path_xml)
test_xml_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 5618.68it/s]

Converting XMLs in folder:  data/test/labels/ ...

-------------> data/test/labels/


,filename,width,height,class,xmin,ymin,xmax,ymax
0,022814asiatodaymasks_960x540.jpg,960,540,bad,319,69,413,133
1,022814asiatodaymasks_960x540.jpg,960,540,good,70,124,176,224
2,022814asiatodaymasks_960x540.jpg,960,540,good,42,285,198,458
3,022814asiatodaymasks_960x540.jpg,960,540,good,334,166,458,297
4,022814asiatodaymasks_960x540.jpg,960,540,good,551,149,824,409


In [64]:
train_xml_df.to_csv(train_path_csv + 'train_labels.csv', index=None)
test_xml_df.to_csv(test_path_csv + 'test_labels.csv', index=None)

print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [65]:
from google.protobuf import text_format
from tensorflow.python.platform import gfile
import io
import pandas as pd
from object_detection.utils import dataset_util


from PIL import Image
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'good':
        return 1
    elif row_label =='bad':
        return 2
    elif row_label =='none': 
        return 3
    return 0

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    """print('group', group)
    print('path', path)
    print('width=', width, 'height=',height)"""

    filename = group.filename.encode('utf8')
    #print('filename', filename)

    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [66]:
train_examples = pd.read_csv(train_path_csv + 'train_labels.csv')
test_examples = pd.read_csv(test_path_csv + 'test_labels.csv')
print(len(train_examples))
print(len(test_examples))

3514
389


In [67]:
train = pd.read_csv("data/train/csv/train_labels.csv")
filenames = list(train.filename.unique())
to_keep = []
for f in filenames:
    if os.path.exists("data/train/images/"+f):
        to_keep.append(f)
train = train[train.filename.isin(to_keep)]
train.to_csv("data/train/csv/train_labels.csv",index=False)

test = pd.read_csv("data/test/csv/test_labels.csv")
filenames = list(test.filename.unique())
to_keep = []
for f in filenames:
    if os.path.exists("data/test/images/"+f):
        to_keep.append(f)
test = test[test.filename.isin(to_keep)]
test.to_csv("data/test/csv/test_labels.csv",index=False)

In [68]:
import tensorflow as tf
from tqdm import tqdm, trange

def generate_tfrecord(csv_file, record_output_path, record_name, path_to_images):
    writer = tf.io.TFRecordWriter(record_output_path + record_name)

    examples = pd.read_csv(csv_file)
    grouped = split(examples, 'filename')
    for group in tqdm(grouped):
        tf_example = create_tf_example(group, path_to_images)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('\nSuccessfully created ', record_name, ': {}'.format(record_output_path))

    return

In [69]:
train_csv_file = train_path_csv + 'train_labels.csv'
generate_tfrecord(train_csv_file, HOME_DIR, 'train.record', train_path_images)

test_csv_file = test_path_csv + 'test_labels.csv'
generate_tfrecord(test_csv_file, HOME_DIR, 'test.record', test_path_images)

 11%|████████▉                                                                          | 7/65 [00:00<00:01, 48.44it/s]


Successfully created  train.record : data/


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:01<00:00, 60.15it/s]



Successfully created  test.record : data/
